<a href="https://colab.research.google.com/github/gugyeoj1n/Natural_Language_Processing/blob/main/10_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv", filename="spam.csv")
data = pd.read_csv('/content/drive/MyDrive/NLP_colab/spam.csv', encoding='latin1')
print('총 샘플의 수 :',len(data))
print(data[:5])

총 샘플의 수 : 5572
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [ ]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])

In [13]:
data.drop_duplicates(subset="v2", inplace=True)

In [14]:
print(data.groupby('v1').size().reset_index(name='cnt'))

   v1   cnt
0   0  4516
1   1   653


In [15]:
X_data, y_data = data['v2'], data['v1']
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, stratify=y_data)

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_encoded = tokenizer.texts_to_sequences(X_train)

In [17]:
word_to_index = tokenizer.word_index
vocab_size = len(word_to_index) + 1
max_len = max(len(l) for l in X_train_encoded)

X_train_padded = pad_sequences(X_train_encoded, maxlen=max_len)

In [20]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train_padded, y_train, epochs=4, batch_size=64, validation_split=0.2)

Epoch 1/4
52/52 [==============================] - 4s 47ms/step - loss: 0.3480 - acc: 0.9027 - val_loss: 0.1786 - val_acc: 0.9686
Epoch 2/4
52/52 [==============================] - 2s 42ms/step - loss: 0.1284 - acc: 0.9683 - val_loss: 0.1057 - val_acc: 0.9686
Epoch 3/4
52/52 [==============================] - 2s 46ms/step - loss: 0.0716 - acc: 0.9810 - val_loss: 0.1027 - val_acc: 0.9674
Epoch 4/4
52/52 [==============================] - 2s 42ms/step - loss: 0.2956 - acc: 0.8894 - val_loss: 1.1129 - val_acc: 0.5163


In [21]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_encoded, maxlen = max_len)
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test_padded, y_test)[1]))

33/33 [==============================] - 1s 19ms/step - loss: 1.2186 - acc: 0.4807

 테스트 정확도: 0.4807
